# **Poređenje klasičnog ML pristupa klasifikaciji teksta, zero shot, i few shot klasifikacije**

Tema rada je poređenje klasifikacije sentimenta tekstualnih recenzija filmova primenom metoda mašinskog učenja, zero shot i few shot.

Predmet rada čine filmske recenzije sa IMDb platforme, koje su predstavljene kroz tekstualni opis i pripadajuću labelu sentimenta. U radu se koristi skraćena verzija popularnog skupa podataka, preuzeta sa platforme Kaggle pod nazivom "IMDB 50K Cleaned Movie Reviews", dostupna na adresi:

https://www.kaggle.com/datasets/ibrahimqasimi/imdb-50k-cleaned-movie-reviews

Izabrana verzija od 5.000 zapisa pruža optimalan balans između reprezentativnosti uzorka za statističku analizu i ekonomičnosti resursa (vreme obrade i utrošak tokena) prilikom rada sa cloud API servisima.

Svrha rada je razvoj i poređenje tri različita pristupa za automatsku klasifikaciju sentimenta: klasičnog mašinskog učenja (SVM), zero-shot klasifikacije i few-shot klasifikacije primenom LLM modela. Poseban fokus je na analizi sposobnosti modela da prepoznaju sentiment uz minimalan broj primera, poredeći robusnost dubokih semantičkih reprezentacija sa generativnim sposobnostima LLM-a.

U okviru metodologije najpre se sprovodi faza čišćenja i pretprocesiranja teksta. Iako odabrani dataset sadrži kolonu sa unapred očišćenim tekstom, ona je u radu odbačena kako bi se sprovelo manuelno čišćenje i osigurala potpuna kontrola nad procesom normalizacije, uklanjanja šuma i lematizacije. BGE koji koristimo svakako sam radi čišćenje teksta, pa bi preterano čišćenje teksta moglo da utiče na njegov rad, ali će se uraditi osnovna obrada i obrada koju BGE model ne radi.

Za numeričku reprezentaciju teksta koristi se savremeni BGE model (BAAI/bge-small-en-v1.5), koji predstavlja vrlo preciznu metodu za generisanje semantičkih embeddinga, s obzirom na to da je potrebno da ML model ima tačnost poredivu sa LLM pristupom. BGE-large bi zahtevao značajno više resursa i vremena i zbog toga nije korišćen. Slični rezultati sa BGE-small verovatno bi mogli da se očejuju i sa SBERT modelom, ali pri korišćenju BGE modela za nas se javlja malo manje posla pri obradi teksta. TF-IDF je takodje bio potencijalna ekonomičnija opcija jer su u pitanju jako duge recenzije, što bi povećalo tačnost ovog modela, mada bi se mogli očekivati blago lošiji rezultati.

Kod few-shot pristupa, selekcija primera u promptu vrši se primenom MMR (Maximal Marginal Relevance) algoritma, čime se obezbeđuje odabir najinformativnijih i najraznovrsnijih primera bez potrebe za manuelnim isprobavanjem različitih kombinacija.

Za zadatak klasifikacije koristi se algoritam Support Vector Machine (SVM) kao primer snažnog klasičnog klasifikatora, dok se za zero i few-shot pristupe koristi LangChain okvir. Tokom rada vrši se podešavanje ključnih hiper-parametara, kao što je regularizacioni parametar C kod SVM-a (radi postizanja optimalne margine razdvajanja u embedding prostoru) i podešavanje temperature LLM-a, uz primenu normalizacije embeddinga pre primene kosinusne sličnosti.

Evaluacija modela sprovodi se primenom standardnih metrika: tačnosti, preciznosti, odziva i F1-score-a. Kao primarna metrika prioritet ima F1-score, jer pruža najobjektivniji uvid u kvalitet klasifikacije kroz balansiranje preciznosti i odziva, što je ključno za donošenje zaključka o superiornosti jednog od poređenih pristupa u realnom kontekstu analize korisničkog zadovoljstva.

Instalacija potrebnih biblioteka i importi

In [ ]:

!pip -q install langchain langchain_community langchain_groq
!pip -q install faiss-cpu
!pip -q install sentence_transformers FlagEmbedding langchain-huggingface
!pip -q install imbalanced-learn spacy
!python -m spacy download en_core_web_lg

In [ ]:
import os
import pandas as pd
import numpy as np
import spacy
import re
from google.colab import userdata, files
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn import metrics
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

Učitavanje podataka

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

Izbacujemo postojeću kolonu 'cleaned_review' jer želimo da uradimo čišćenje sami (manuelno), kako bismo osigurali kvalitet podataka, kao što smo planirali.

In [ ]:
if 'cleaned_review' in df.columns:
    df.drop(columns=['cleaned_review'], inplace=True)

Pregled dataseta

In [ ]:
print(df.head())

Provera izbalansiranosti, ipak ovo jeste skraćena verzija originala. Imamo otprilike isti broj pozitivnih i negativnih i to je dobar znak.

In [ ]:
print(df['sentiment'].value_counts())

Provera duplikata i null vrednosti

In [ ]:
print(f"Duplikati: {df.duplicated().sum()}")

In [ ]:
df = df.drop_duplicates()

In [ ]:
print(f"Duplikati: {df.duplicated().sum()}")

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
pd.set_option('display.max_colwidth', 2000)
df.sample(5)

Manuelno preprocesiranje

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
def minimal_clean_text(text):
    # Uklanjanje HTML tagova
    text = re.sub(r'<.*?>', ' ', text)

    # višestrukih razmaka i novih redova
    text = re.sub(r'\s+', ' ', text).strip()


    return text

In [ ]:
df['manual_cleaned_review'] = df['review'].apply(minimal_clean_text)
df.head()

In [ ]:
df['review_len'] = df['manual_cleaned_review'].astype(str).str.len()

In [ ]:
df['review_len'].describe()

BGE Embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

model_name = "BAAI/bge-small-en-v1.5"

In [ ]:
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

Generisanje vektora

In [ ]:
X_embeddings = bge_embeddings.embed_documents(df['manual_cleaned_review'].tolist())
X_embeddings = np.array(X_embeddings)

In [ ]:
print(f"Embedding gotov. Oblik matrice: {X_embeddings.shape}")

Split podataka i priprema tabele

In [ ]:
Random = 10

In [ ]:
y = df['sentiment'].map({'positive': 1, 'negative': 0}).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_embeddings, y, test_size=0.2, random_state=Random, stratify=y
)

In [ ]:
print(f"Podaci podeljeni. Train: {len(X_train)}, Test: {len(X_test)}")

Trening SVM modela sa tuning-om

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
svm_model = SVC(kernel='linear', random_state=Random)

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100]
}

grid_search = GridSearchCV(
    svm_model,
    param_grid,
    scoring='f1', # Fokusiramo se na F1-score kao primarnu metriku
    cv=5,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_svm = grid_search.best_estimator_
print(f"Najbolji parametri: {grid_search.best_params_}")
print(f"Najbolji F1 rezultat na treningu: {grid_search.best_score_:.4f}")

Evaluacija klasičnog modela

In [ ]:
y_pred = best_svm.predict(X_test)

In [ ]:
print("\nREZULTATI ZA KLASIČAN ML (SVM + BGE):")
print(metrics.classification_report(y_test, y_pred, target_names=['negative', 'positive']))

In [ ]:
balanced_acc = metrics.balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

# Čuvamo rezultate za kasnije poređenje sa LLM
svm_results = metrics.classification_report(y_test, y_pred, output_dict=True)

Setup za LLM (Groq i LangChain)

In [ ]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from time import sleep


In [ ]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Inicijalizacija modela preko
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0 # 0.0 za stabilne, determinističke odgovore
)

 Priprema uzoraka

In [ ]:
ex_pool_df = df.sample(100, random_state=Random)

In [ ]:
# Izbacujemo ove redove iz glavnog dataseta da ne bi došlo do "curenja" (leakage)
# podataka u testni set
eval_pool_df = df.drop(ex_pool_df.index)

In [ ]:
# Sada uzimamo 100 nasumičnih recenzija za finalnu evaluaciju LLM-a
eval_df = eval_pool_df.sample(100, random_state=Random)

In [ ]:
# Priprema eval_df za rad
eval_df = eval_df[['manual_cleaned_review', 'sentiment']].rename(columns={'manual_cleaned_review': 'review', 'actual': 'sentiment'})
# Dodajemo kolonu 'actual'
eval_df['actual'] = eval_df['sentiment']

print(f"Baza primera za MMR spremna ({len(ex_pool_df)} redova).")
print(f"Set za evaluaciju LLM-a spreman ({len(eval_df)} redova).")

Zero-shot Klasifikacija

In [ ]:
zero_shot_system_msg = """
You are an expert film critic and NLP specialist.
You are a binary sentiment classifier. You MUST respond with ONLY one word: 'positive' or 'negative'. No preamble, no explanation. No preamble, no explanation, no 'mixed'.
Review to classify:
"""

zero_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", zero_shot_system_msg),
    ("user", "{input}")
])

In [ ]:
zero_shot_chain = zero_shot_prompt | llm | StrOutputParser()

In [ ]:
preds = []
print(f"Započinjem Zero-shot klasifikaciju za {len(eval_df)} primera...")
for i, text in enumerate(eval_df['review']):
    print(f"{i+1}/100")

    try:
        res = zero_shot_chain.invoke({"input": text}).strip().lower()
    except Exception as e:
        print("Greška:", e)
        res = "none"

    preds.append(res)
    sleep(5)

print("Zero-shot završen.")
eval_df['zero_shot_pred'] = preds

Few-shot Klasifikacija

In [ ]:
from langchain_core.example_selectors.semantic_similarity import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

Pošto polovina recenzija ima dužinu preko 1000 karaktera, obrada bi trajala predugo, pa ćemo ih skratiti na maksimalno 1000. To bi trebalo da bude dovoljno da pokaže sentiment.

In [ ]:
def truncate_text(text, limit=1000):
    return text[:limit] + "..." if len(text) > limit else text

In [ ]:
# Transformišemo ex_pool_df u listu rečnika za MMR
examples = [
    {"requirement": truncate_text(row['manual_cleaned_review']), "type": row['sentiment']}
    for _, row in ex_pool_df.iterrows()
]

In [ ]:
# MMR Selekcija primera koristeći BGE embeddings
mmr_example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=bge_embeddings,
    vectorstore_cls=FAISS,
    k=3 # Broj primera u promptu
)

In [ ]:
few_shot_prompt_template = FewShotChatMessagePromptTemplate(
    example_selector=mmr_example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("user", "{requirement}"),
        ("ai", "{type}"),
    ]),
    input_variables=["input"]
)

In [ ]:
final_few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert film critic and NLP specialist. You are a binary sentiment classifier. You MUST respond with ONLY one word: 'positive' or 'negative'. No preamble, no explanation, no 'mixed'."),
    few_shot_prompt_template,
    ("user", "Classification task: Respond with 'positive' or 'negative' ONLY.\nReview:\n{input}")
])

In [ ]:
few_shot_chain = final_few_shot_prompt | llm | StrOutputParser()

In [ ]:
few_shot_preds = []
for i, text in enumerate(eval_df['review']):
    try:
        # Skraćujemo ulazni tekst da uštedimo tokene
        short_text = truncate_text(text)
        res = few_shot_chain.invoke({"input": short_text}).strip().lower()
        few_shot_preds.append(res)

        # Progres bar
        if (i+1) % 5 == 0:
            print(f"Obrađeno {i+1}/100...")


    except Exception as e:
        print(f"Greška na indeksu {i}: {e}")
        few_shot_preds.append("none")
    sleep(3)


eval_df['few_shot_pred'] = few_shot_preds
print("Few-shot završen.")

Finalno poređenje

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
##print("\nZERO-SHOT REZULTATI:")
#print(classification_report(eval_df['actual'], eval_df['zero_shot_pred']))

#print("\nFEW-SHOT (MMR) REZULTATI:")
#print(classification_report(eval_df['actual'], eval_df['few_shot_pred']))

In [ ]:
#Funkcija za izvlačenje ključnih metrika
def get_model_metrics(y_true, y_pred, model_name):
    # Generisanje reporta kao dikta
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    # Dodatne metrike
    b_acc = balanced_accuracy_score(y_true, y_pred)
    macro_avg = report.get('macro avg', {})
    weighted_avg = report.get('weighted avg', {})

    return {
        'Model': model_name,
        'Accuracy': report.get('accuracy', 0),
        'Macro Precision': macro_avg.get('precision', 0),
        'Macro Recall': macro_avg.get('recall', 0),
        'Macro F1-Score': macro_avg.get('f1-score', 0),
        'Weighted Precision': weighted_avg.get('precision', 0),
        'Weighted Recall': weighted_avg.get('recall', 0),
        'Weighted F1-Score': weighted_avg.get('f1-score', 0),
        'Balanced Accuracy': b_acc
    }

def plot_cm(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=['positive', 'negative'])
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['positive', 'negative'],
                yticklabels=['positive', 'negative'])
    plt.xlabel('Predviđeno')
    plt.ylabel('Stvarno')
    plt.title(title)
    plt.show()

In [ ]:
#Prikupljanje svih rezultata
all_results = []

In [ ]:
# Dodajemo SVM metrike
# Pretvaramo 0/1 u labele za konzistentnost sa matricom
svm_y_true_labels = pd.Series(y_test).map({1: 'positive', 0: 'negative'})
svm_y_pred_labels = pd.Series(y_pred).map({1: 'positive', 0: 'negative'})
all_results.append(get_model_metrics(svm_y_true_labels, svm_y_pred_labels, "SVM + BGE (Classical)"))

# Dodajemo LLM metrike (iz eval_df - 100 redova)
all_results.append(get_model_metrics(eval_df['actual'], eval_df['zero_shot_pred'], "LLM Zero-shot"))
all_results.append(get_model_metrics(eval_df['actual'], eval_df['few_shot_pred'], "LLM Few-shot"))

# 3) Izgradnja i formatiranje tabele
all_results_df = pd.DataFrame(all_results).set_index("Model")
all_results_df = all_results_df.apply(lambda col: pd.to_numeric(col, errors='ignore').round(4))
all_results_df = all_results_df.sort_values(by="Weighted F1-Score", ascending=False)

print("\n--- FINALNO POREĐENJE PERFORMANSI ---")
print(all_results_df)



In [ ]:
# Matrice konfuzije za vizuelni pregled
print("\n--- MATRICE KONFUZIJE ---")
plot_cm(eval_df['actual'], eval_df['zero_shot_pred'], "Zero-shot Matrica")
plot_cm(eval_df['actual'], eval_df['few_shot_pred'], "Few-shot (MMR) Matrica")
plot_cm(svm_y_true_labels, svm_y_pred_labels, "SVM + BGE Matrica (na punom test setu)")

# Interpretacija rezultata
Na osnovu sprovedenog eksperimenta nad skupom podataka IMDb filmskih recenzija (5.000 zapisa), izvršena je evaluacija tri različita pristupa klasifikaciji sentimenta. Rezultati pokazuju visok stepen uspešnosti svih modela, sa specifičnim uvidima u prednosti i mane svake metodologije.

1. Few-shot

Inicijalni testovi sa 5 primera (k=5) i dužim tekstovima pokazivali su blagi pad performansi zbog "šuma" u kontekstu. Međutim, optimizacijom na 3 primera i uvođenjem ograničenja od 1000 karaktera, Few-shot model je podigao svoju tačnost na 94.0%, čime se potpuno izjednačio sa Zero-shot pristupom.

Zaključak: Ovo dokazuje princip "manje je više" u inženjeringu promptova. Skraćivanje primera na ključnih 1000 karaktera pomoglo je modelu Llama 3.1 da zadrži fokus na suštini sentimenta, dok je smanjenje broja na 3 optimalno balansiralo između instrukcije i ilustracije.

2. Zero-shot

Oba LLM pristupa postigla su identičan rezultat (94.0% Accuracy, 0.9483 Balanced Accuracy).

Analiza matrica konfuzije: Interesantno je da obe verzije LLM-a imaju identičnu matricu konfuzije ($52/6/0/42$). Oba modela su nepogrešiva pri detekciji negativnog sentimenta (0 grešaka), dok pokazuju blagu "strogoću" kod pozitivnih recenzija (6 promašaja).
Uvid: Činjenica da primeri nisu uspeli da nadmaše čistu instrukciju sugeriše da Llama 3.1 poseduje toliko snažno predznanje o binarnom sentimentu da su primeri služili samo kao potvrda već usvojenih pravila, a ne kao izvor novog znanja

3. SVM + BGE-small

Klasičan ML model (SVM sa BGE-small embeddingzima) ostvario je izuzetnih 93.8% tačnosti na desetostruko većem testnom setu (1.000 uzoraka).

Značaj: Dok LLM briljira na malim uzorcima, SVM demonstrira neverovatnu stabilnost i konzistentnost na velikoj skali. Minimalna razlika od svega 0.2% u odnosu na LLM dokazuje da je kombinacija SVM-a i modernih embeddinga (čak i small varijante) i dalje najisplativije rešenje za masovnu produkcionu obradu podataka.


**Zaključak i poređenje**

Glavni doprinos rada: Istraživanje je pokazalo da se vrhunski rezultati (preko 93%) mogu postići na tri različita načina. Ključni naučni uvid je da se LLM modeli mogu optimizovati smanjenjem količine informacija u promptu, dok tradicionalni algoritmi (SVM) uz adekvatne vektorske reprezentacije (BGE) ne zaostaju za najmodernijim generativnim modelima u preciznosti klasifikacije.